# EDA analysis of CMAPSS dataset

### Dataset Overview

| Dataset | Train Trajectories | Test Trajectories | Operating Conditions | Fault Modes                             |
|---------|--------------------|-------------------|----------------------|------------------------------------------|
| FD001   | 100                | 100               | 1 (Sea Level)        | 1 (HPC Degradation)                      |
| FD002   | 260                | 259               | 6                    | 1 (HPC Degradation)                      |
| FD003   | 100                | 100               | 1 (Sea Level)        | 2 (HPC Degradation, Fan Degradation)     |
| FD004   | 248                | 249               | 6                    | 2 (HPC Degradation, Fan Degradation)     |


### Import libraries

In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_dir = "../data/"

### Import data 

In [54]:
# Standard inputs
column_names = [
    "unit", "cycles", "op_1", "op_2", "op_3",
]
# Append sensor columns
colnum = 22
for i in range(1,colnum): 
    column_names.append( "s_%s"%(i))

In [62]:
train = pd.read_csv(data_dir + 'train_FD001.txt',sep=' ', header=None)
train = train.dropna(axis = 1)
train.columns = column_names[:train.shape[1]]

test = pd.read_csv(data_dir + 'test_FD001.txt',sep=' ', header=None)
test = test.dropna(axis = 1)
test.columns = column_names[:test.shape[1]]

y_test = pd.read_csv(data_dir + 'RUL_FD001.txt', sep=' ', header=None, names=['RUL'])

eng_num = train["unit"].nunique()
print( "Number of units is %s"%(eng_num))

Number of units is 100


#### Get more information on the data

In [64]:
train.describe()

,unit,cycles,op_1,op_2,s_1,s_2,s_3,s_4,s_5,s_6,...,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21
count,20631.000000,20631.000000,20631.000000,20631.000000,20631.00,20631.000000,20631.000000,20631.000000,2.063100e+04,20631.000000,...,20631.000000,20631.000000,20631.000000,20631.000000,2.063100e+04,20631.000000,20631.0,20631.0,20631.000000,20631.000000
mean,51.506568,108.807862,-0.000009,0.000002,518.67,642.680934,1590.523119,1408.933782,1.462000e+01,21.609803,...,521.413470,2388.096152,8143.752722,8.442146,3.000000e-02,393.210654,2388.0,100.0,38.816271,23.289705
std,29.227633,68.880990,0.002187,0.000293,0.00,0.500053,6.131150,9.000605,5.329200e-15,0.001389,...,0.737553,0.071919,19.076176,0.037505,3.469531e-18,1.548763,0.0,0.0,0.180746,0.108251
min,1.000000,1.000000,-0.008700,-0.000600,518.67,641.210000,1571.040000,1382.250000,1.462000e+01,21.600000,...,518.690000,2387.880000,8099.940000,8.324900,3.000000e-02,388.000000,2388.0,100.0,38.140000,22.894200
25%,26.000000,52.000000,-0.001500,-0.000200,518.67,642.325000,1586.260000,1402.360000,1.462000e+01,21.610000,...,520.960000,2388.040000,8133.245000,8.414900,3.000000e-02,392.000000,2388.0,100.0,38.700000,23.221800
50%,52.000000,104.000000,0.000000,0.000000,518.67,642.640000,1590.100000,1408.040000,1.462000e+01,21.610000,...,521.480000,2388.090000,8140.540000,8.438900,3.000000e-02,393.000000,2388.0,100.0,38.830000,23.297900
75%,77.000000,156.000000,0.001500,0.000300,518.67,643.000000,1594.380000,1414.555000,1.462000e+01,21.610000,...,521.950000,2388.140000,8148.310000,8.465600,3.000000e-02,394.000000,2388.0,100.0,38.950000,23.366800
max,100.000000,362.000000,0.008700,0.000600,518.67,644.530000,1616.910000,1441.490000,1.462000e+01,21.610000,...,523.380000,2388.560000,8293.720000,8.584800,3.000000e-02,400.000000,2388.0,100.0,39.430000,23.618400


In [ ]:
# Drop operating setting 3 since its value is not changing
train = train.drop("op_3", axis = 1)
train.head(5)

,unit,cycles,op_1,op_2,s_1,s_2,s_3,s_4,s_5,s_6,...,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21
0,1,1,-0.0007,-0.0004,518.67,641.82,1589.70,1400.60,14.62,21.61,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,518.67,642.15,1591.82,1403.14,14.62,21.61,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,518.67,642.35,1587.99,1404.20,14.62,21.61,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,518.67,642.35,1582.79,1401.87,14.62,21.61,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,518.67,642.37,1582.85,1406.22,14.62,21.61,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


In [51]:
# for i in range(3):
#     engine_id = i + 1
#     df_subset = train[train["unit"] == engine_id]
#     plt.plot(df_subset["cycles"], df_subset["s_3"], label=f"Engine {engine_id}")
# plt.xlabel("Cycle")
# plt.ylabel("Sensor 2")
# plt.title("Sensor Trend")
# plt.legend()